In [1]:
pip install datasets

In [2]:
pip install evaluate

In [3]:
# classification(comments)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from transformers import Trainer, TrainingArguments, pipeline, AutoTokenizer, AutoModelForMaskedLM
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
from datasets import Dataset


In [5]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP Project/NLP Dataset/sentences/darija_clean.csv')
df1=df.copy()
df2=df.copy()

In [6]:
df1

,cleaned_text,label,sentence_length
0,الي كيقولوا الشرع اعطاه اربعة وهو محامي عاد وم...,positive,967
1,الي كيقولوا الشرع اعطاه اربعة وهو محامي عاد وم...,neutral,967
2,واش هادا كايبغيك باله عليك لي كايهدر معاك بهاد...,negative,993
3,فظائح ومهازل والشعب التونسي يعاني البطالة والف...,Negative,983
4,يعطيه الصحة على الأقل لقينا واحد فاهم الحكاية ...,Positive,883
...,...,...,...
43693,Ya ta7an,Negative,8
43694,أنا أؤمن,neutral,8
43695,كيدايرة؟,Neutral,8
43696,اول لايك,neutral,8


In [7]:
# Step 1: Split the data into 80% train and 20% test
train_df, test_df = train_test_split(df1, test_size=0.2, random_state=42)

# Step 2: Split the 80% training set into 65% train and 15% validation
train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 29714
Validation set size: 5244
Test set size: 8740


In [8]:
# Load DarijaBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("SI2M-Lab/DarijaBERT")
model = AutoModelForMaskedLM.from_pretrained("SI2M-Lab/DarijaBERT")

classification = pipeline(task="text-classification", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
BertForMaskedLM has generative capabilities,

In [9]:
dataset = Dataset.from_pandas(df)
# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["cleaned_text"], padding="max_length", truncation=True, max_length=512)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/43698 [00:00<?, ? examples/s]

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch

# Charger le tokenizer et le modèle DarijaBERT
tokenizer = AutoTokenizer.from_pretrained("SI2M-Lab/DarijaBERT")
model = AutoModel.from_pretrained("SI2M-Lab/DarijaBERT")

# Exemple de phrases en darija
sentences = [
    "السلام عليكم",
   "شكون لي غادي يجي معانا؟",
    "أنا ذاهب إلى السوق",
    "الجو جميل اليوم",
    "هل تريد المساعدة؟",
    "حنا غاديين نشوفو الفيلم",
    "راك غالط فهاد القضية"

]

# Générer les embeddings
def generate_embeddings(sentences):
    embeddings_list = []
    for sentence in sentences:
        # Tokenisation
        inputs = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        # Génération des embeddings
        with torch.no_grad():
            outputs = model(**inputs)
        # Récupérer l'embedding pour la phrase (utiliser le token [CLS])
        sentence_embedding = outputs.last_hidden_state[:, 0, :].squeeze().tolist()  # [CLS] token embedding
        embeddings_list.append(sentence_embedding)
    return embeddings_list

# Appeler la fonction et afficher les résultats
embeddings = generate_embeddings(sentences)

# Affichage des embeddings
for i, embedding in enumerate(embeddings):
    print(f"Phrase {i+1}: {sentences[i]}")
    print(f"Embedding (5 premières dimensions): {embedding[:5]}")
    print("-" * 50)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Some weights of BertModel were not initializ

Phrase 1: السلام عليكم
Embedding (5 premières dimensions): [0.16177131235599518, 1.2530937194824219, -0.002813718281686306, -1.2660000324249268, -1.229274868965149]
--------------------------------------------------
Phrase 2: شكون لي غادي يجي معانا؟
Embedding (5 premières dimensions): [-0.5465778708457947, 0.3349747657775879, -0.053129199892282486, -0.10825037956237793, -0.9749795198440552]
--------------------------------------------------
Phrase 3: أنا ذاهب إلى السوق
Embedding (5 premières dimensions): [-0.8887969255447388, -0.6956924200057983, -0.8948899507522583, -1.2390978336334229, -1.2929633855819702]
--------------------------------------------------
Phrase 4: الجو جميل اليوم
Embedding (5 premières dimensions): [-2.05435848236084, 0.028445400297641754, -0.4740184247493744, -1.1570290327072144, -1.8838459253311157]
--------------------------------------------------
Phrase 5: هل تريد المساعدة؟
Embedding (5 premières dimensions): [-0.3798559308052063, 1.3538665771484375, -0.870304

In [ ]:
# Extract embeddings
def extract_embeddings(batch):
    inputs = {
        "input_ids": torch.tensor(batch["input_ids"]),
        "attention_mask": torch.tensor(batch["attention_mask"]),
    }
    with torch.no_grad():
        outputs = model(**inputs)
        token_embeddings = outputs.last_hidden_state  # Shape: (batch_size, seq_length, hidden_size)
        sentence_embeddings = token_embeddings[:, 0, :]  # `[CLS]` token embedding
        return {"embeddings": sentence_embeddings.tolist()}  # Convert to list for storage

# Apply embedding extraction
embedded_dataset = tokenized_dataset.map(extract_embeddings, batched=True)

# Example: Access embeddings
example = embedded_dataset[0]
print("Text:", example["text"])
print("Embedding (first 5 dimensions):", example["embeddings"][:5])

Map:   0%|          | 0/43698 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format(type='torch', columns=['embeddings', 'labels'])
val_dataset.set_format(type='torch', columns=['embeddings', 'labels'])
test_dataset.set_format(type='torch', columns=['embeddings', 'labels'])